In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os

In [6]:
warehouse_path = os.path.abspath("data/warehouse")
print(warehouse_path)

spark = SparkSession.builder \
    .appName("tests") \
    .config("spark.hadoop.hadoop.native.lib", "false") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.0") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", warehouse_path) \
    .getOrCreate()

def test_table_and_column_comments():
    tables_df = spark.sql("SHOW TABLES IN local.bronze")\
    .withColumn("fullTableName", F.concat_ws(".", F.col("nameSpace"), F.col("tableName")))\
    .select("fullTableName")


    tables = [row["fullTableName"] for row in tables_df.collect()]
    
    for table in tables:
        full_table = f"local.{table}"
        
        # Check for table comment
        table_info = spark.sql(f"DESCRIBE TABLE EXTENDED {full_table}").collect()
        comment_line = [row for row in table_info if row.col_name == "Comment"]
        assert comment_line and comment_line[0].data_type.strip(), f"Missing table comment for {full_table}"

        # Check for column comments
        table_column_info = spark.sql(f"DESCRIBE TABLE {full_table}").collect()
        column_names = [row.col_name for row in table_column_info if row.col_name and not row.col_name.startswith("#")]
        for row in table_info:
            if row.data_type != "" and row.col_name in column_names:
                assert row.comment and row.comment.strip(), f"Missing column comment for {full_table}.{row.col_name}"

    print("✅ All tables and columns have comments.")


c:\Users\joonas.syrjanen\Documents\Data rag\data\warehouse


In [7]:
test_table_and_column_comments()

AssertionError: Missing column comment for local.bronze.p__l_march_2021.Ajio_MRP

In [4]:
spark.sql("SHOW TABLES IN local.bronze")\
    .withColumn("fullTableName", F.concat_ws(".", F.col("nameSpace"), F.col("tableName")))\
    .select("fullTableName").show()

+--------------------+
|       fullTableName|
+--------------------+
|        bronze.may22|
| bronze.expense_iigf|
|bronze.internatio...|
|bronze.p__l_march...|
|bronze.amazon_sal...|
|  bronze.sale_report|
|bronze.cloud_ware...|
+--------------------+



In [4]:
test_table_and_column_comments()

AssertionError: Missing column comment for local.bronze.may22.index

In [3]:
spark.sql("select * from local.bronze.may22").show()

+--------------+----------+-------+--------+------+---+-------+-------------+--------+----------+--------------+------------+------------+----------+---------+------------+
|           Sku|  Style_Id|Catalog|Category|Weight| TP|MRP_Old|Final_MRP_Old|Ajio_MRP|Amazon_MRP|Amazon_FBA_MRP|Flipkart_MRP|Limeroad_MRP|Myntra_MRP|Paytm_MRP|Snapdeal_MRP|
+--------------+----------+-------+--------+------+---+-------+-------------+--------+----------+--------------+------------+------------+----------+---------+------------+
|  Os206_3141_S|Os206_3141|Moments|   Kurta|   0.3|538|   2178|         2295|    2295|      2295|          2295|        2295|        2295|      2295|     2295|        2295|
|  Os206_3141_M|Os206_3141|Moments|   Kurta|   0.3|538|   2178|         2295|    2295|      2295|          2295|        2295|        2295|      2295|     2295|        2295|
|  Os206_3141_L|Os206_3141|Moments|   Kurta|   0.3|538|   2178|         2295|    2295|      2295|          2295|        2295|        22

In [7]:
if "snap-7081952170530623771-1-7c343840-7e97-46d6-a4f2-25642a3ae142.avro" == "snap-7081952170530623771-1-7c343840-7e97-46d6-a4f2-25642a3ae142.avro":
    print("File name matches expected pattern.")
else:
    print("File name does not match expected pattern.")

File name matches expected pattern.


In [9]:
spark.read.format("iceberg").load("local.bronze.amazon_sale_report").show()

Py4JJavaError: An error occurred while calling o52.showString.
: org.apache.iceberg.exceptions.NotFoundException: Failed to open input stream for file: data/warehouse/bronze/amazon_sale_report/metadata/snap-7081952170530623771-1-7c343840-7e97-46d6-a4f2-25642a3ae142.avro
	at org.apache.iceberg.hadoop.HadoopInputFile.newStream(HadoopInputFile.java:185)
	at org.apache.iceberg.avro.AvroIterable.newFileReader(AvroIterable.java:102)
	at org.apache.iceberg.avro.AvroIterable.iterator(AvroIterable.java:77)
	at org.apache.iceberg.avro.AvroIterable.iterator(AvroIterable.java:37)
	at org.apache.iceberg.relocated.com.google.common.collect.Iterables.addAll(Iterables.java:332)
	at org.apache.iceberg.relocated.com.google.common.collect.Lists.newLinkedList(Lists.java:261)
	at org.apache.iceberg.ManifestLists.read(ManifestLists.java:42)
	at org.apache.iceberg.BaseSnapshot.cacheManifests(BaseSnapshot.java:176)
	at org.apache.iceberg.BaseSnapshot.deleteManifests(BaseSnapshot.java:210)
	at org.ap

In [8]:
from py4j.java_gateway import java_import

java_import(spark._jvm, 'org.apache.hadoop.fs.Path')
path = spark._jvm.Path(spark.conf.get("spark.sql.catalog.local.warehouse"))
fs = path.getFileSystem(spark._jsc.hadoopConfiguration())
print("Resolved warehouse path:", path.makeQualified(fs.getUri(), fs.getWorkingDirectory()))


Resolved warehouse path: file:/c:/Users/joonas.syrjanen/Documents/Data rag/data/warehouse
